# Bayes Tree Strategy Prediction

In [1]:
import pandas as pd
import numpy as np
import copy

import MySQLdb.cursors
import MySQLdb

import bayes_tree # contains bayes tree class
from bayes_tree import tree
import config as cfg # contains unit lookups/helper functions/credentials

## Get Test Set and Prepare MySQL Cursors

In [2]:
database_list = ['sc_pvp','sc_pvt','tvt']  # 'sc_pvz', 'tvz','zvz'
connection_list = []
cursor_list = []
for database in database_list:
    db = MySQLdb.connect(
    host = cfg.credentials['host'],
    user = cfg.credentials['user'],
    passwd = cfg.credentials['passwd'],
    database = database,
    cursorclass = MySQLdb.cursors.SSCursor)
    connection_list.append(db)
    cursor_list.append(db.cursor())

In [3]:
!ls pickles

blind_prob.pickle        obs_id_list.pickle       raw_prediction.pickle
cleaned_obs.pickle       player_races.pickle      raw_train_builds.pickle
clustered_replays.pickle raw_id_list.pickle       test_replays.pickle
memo_tree.pickle         raw_obs.pickle           unit_counts.pickle


In [4]:
test_replays = cfg.from_pickle('pickles/test_replays.pickle')

In [5]:
test_replays.head()

,cluster,race,playerreplayID,cursor
"(sc_pvt, 4698)",1,1,4698,sc_pvt
"(sc_pvt, 3988)",7,2,3988,sc_pvt
"(sc_pvt, 2546)",9,2,2546,sc_pvt
"(sc_pvp, 1248)",8,2,1248,sc_pvp
"(sc_pvt, 1429)",6,1,1429,sc_pvt


## Generate Observation Logs

In [6]:
def get_obs(playerreplayIDs,time,cursor):
    """
    Requests raw visibility states for a set of player-replay ids
    ---
    Inputs:
    playerreplayIDs: List of ints. List of replays to get data for - note that IDs are repeated across database
    time: Integer. Represents gametime in frames. Will get all visibility state changes up until cutoff
    cursor: connected MySQLdb cursor object. SS cursor has better performance.
    """
    
    # chunking large requests into smaller ones to reduce request failure
    n = 300
    results = []
    request_chunks = [playerreplayIDs[i * n:(i + 1) * n] for i in range((len(playerreplayIDs) + n - 1) // n )]
    for segment in request_chunks:
        playerreplayIDs_string = ",".join(str(x) for x in segment)
        query = """
            SELECT PlayerReplayID, visibilitychange.UnitID, UnitTypeID, ChangeVal,CAST(ChangeTime / 24 AS UNSIGNED) 
            FROM visibilitychange 
            LEFT JOIN unit ON visibilitychange.UnitID = unit.UnitID
            WHERE visibilitychange.ViewerID != unit.PlayerReplayID AND PlayerReplayID in ({}) AND ChangeTime <= {}
            ORDER BY ChangeTime, visibilitychange.UnitID;
        """.format(playerreplayIDs_string,time)
        print(query)
    
        cursor.execute(query)
        results += cursor.fetchall()
        
    print('done!')
    ids = set(map(lambda x:x[0], results))
    results_by_ids = [[y[1:] for y in results if y[0]==x] for x in ids]
    
    return results_by_ids,ids

def process_obs(results):
    """
    Converts list of visibility states into a list of observations of unit_types with time stamps
    ---
    Inputs:
    results: list of visibility state changes by replay
    """
    obs_list_list = []
    
    for result in results: # note that we are ticking through observations through sequential order
        
        # buildings have to be treated differently than units because it is assumed we know each unique building by 
        # its location, unlike units which can move around
        # units of the same type must be visibile at the same time to confirm multiple units of the same type
        building_uniques = {}
        unit_memory = {} # keeps track of units we currently "see"
        obs_list = []
        
        for obs in result:
            
            # building logic
            if obs[1] in cfg.buildings.keys():
                if obs[1] in building_uniques.keys():
                    if obs[0] in building_uniques[obs[1]]:
                        continue
                    building_uniques[obs[1]].append(obs[0])
                else:
                    building_uniques[obs[1]] = [obs[0]]
                
                obs_list.append( ( (obs[1],len(building_uniques[obs[1]])) ,obs[3]) )
                
            # unit logic    
            else:
                if obs[2] == 1: # 1 means that the unit is seen
                    if obs[1] in unit_memory.keys():
                        if obs[0] in unit_memory[obs[1]]:
                            continue
                        unit_memory[obs[1]].append(obs[0])
                    else:
                        unit_memory[obs[1]] = [obs[0]]
                else:
                    unit_memory[obs[1]].remove(obs[0])
                    
                current_count = len(unit_memory[obs[1]])
                    
                prev_max = max([x[0][1] for x in obs_list if x[0][0]==obs[1]]+[0])
                if current_count > prev_max:
                    obs_list.append( ( (obs[1],current_count) ,obs[3]) )
                    
        obs_list_list.append(obs_list)
                    
    return obs_list_list

In [7]:
try:
    raw_obs = cfg.from_pickle('pickles/raw_obs.pickle')
    obs_id_list = cfg.from_pickle('pickles/obs_id_list.pickle')
except:
    raw_obs = []
    obs_id_list = []
    for index,database in enumerate(database_list):
        playerreplayIDs = test_replays['playerreplayID'][test_replays['cursor'] == database]
        cursor = cursor_list[index]
        builds, ids = get_obs(playerreplayIDs,cfg.TIME-30*24,cursor)
        raw_obs.append(builds)
        obs_id_list.append(ids)
    
    cfg.to_pickle(raw_obs,'raw_obs')
    cfg.to_pickle(obs_id_list,'obs_id_list')

In [8]:
processed_obs = []
for obs in raw_obs:
    processed_obs.append(process_obs(obs))

In [9]:
all_obs = []
all_ids = []
all_unique_ids = []
all_races = []
for matchup_ind,matchup in enumerate(processed_obs):
    for match_ind,match in enumerate(matchup):
        
        live_id = list(obs_id_list[matchup_ind])[match_ind]
        all_ids.append(live_id)
        unique_id = (database_list[matchup_ind],live_id)
        all_unique_ids.append(unique_id)
        
        race = test_replays['race'][test_replays.index == unique_id].values[0]
        all_races.append(race)
           
        all_obs.append(match)

In [10]:
cfg.to_pickle([all_obs,all_unique_ids],'cleaned_obs')

## Blind Guess

In [11]:
tree = cfg.from_pickle('pickles/memo_tree.pickle')

In [12]:
race_order = [2,1,0]
blind_prob = {}
for index,node in enumerate(tree.root.children):
    active_clusters = node.cluster
    total = 0
    for cluster in active_clusters.values():
        total += cluster
        
    for key in active_clusters.keys():
        active_clusters[key] = active_clusters[key] / total
    
    blind_prob[race_order[index]] = active_clusters

In [13]:
cfg.to_pickle(blind_prob,'blind_prob')

In [14]:
results_df = test_replays[['cluster','race']].copy()

In [15]:
results_df['blind'] = results_df.apply(lambda x: blind_prob[x[1]][x[0]],axis=1)

In [16]:
results_df.head()

,cluster,race,blind
"(sc_pvt, 4698)",1,1,0.142632
"(sc_pvt, 3988)",7,2,0.441491
"(sc_pvt, 2546)",9,2,0.174137
"(sc_pvp, 1248)",8,2,0.239191
"(sc_pvt, 1429)",6,1,0.144996


## Guess with Observations

In [17]:
raw_prediction = []
for index,live_id in enumerate(all_unique_ids):
    print(f'Working on {index+1}')
    live_obs = all_obs[index]
    num_obs = len(live_obs)
    tree.reset()
    for obs in live_obs:
        tree.update_tree(obs)
    
    print('-------------------------------')
    raw_prediction.append([live_id,num_obs,tree.predict(cfg.TIME-30*24,'cluster')])

Working on 1
Freq Check: 0.9999999999999943
Freq Check: 1.000000000000012
Freq Check: 0.9999999999999772
Freq Check: 0.999999999999997
Freq Check: 0.9999999999999943
Freq Check: 1.0000000000000058
Freq Check: 0.9999999999999942
Freq Check: 1.0000000000000058
Freq Check: 0.9999999999999942
Freq Check: 1.0000000000000058
Freq Check: 0.9999999999999944
Freq Check: 1.0000000000000109
Freq Check: 0.9999999999999943
Freq Check: 1.0000000000000187
Freq Check: 0.9999999999999929
Freq Check: 1.0000000000000107
Freq Check: 1.00000000000001
Freq Check: 0.9999999999999741
Freq Check: 0.9999999999999452
Freq Check: 1.0000000000000542
Freq Check: 0.9999999999999458
Freq Check: 0.9999999999999042
Freq Check: 1.0000000000001048
Freq Check: 0.9999999999992019
Freq Check: 1.000000000000593
Freq Check: 0.9999999999992559
Freq Check: 1.0000000000009315
Freq Check: 0.9999999999982673
Freq Check: 1.0000000000017324
Freq Check: 0.9999999999982689
Freq Check: 1.0000000000016651
Freq Check: 0.9999999999982139


Freq Check: 0.9999999999999579
Freq Check: 0.9999999999999487
Freq Check: 1.0000000000000548
Freq Check: 0.999999999999983
Freq Check: 0.9999999999999934
Freq Check: 1.0000000000000067
Freq Check: 1.0000000000000804
Freq Check: 0.9999999999998325
Freq Check: 1.0000000000001676
Freq Check: 0.999999999999872
Freq Check: 1.000000000000087
Freq Check: 0.9999999999998042
Freq Check: 1.0000000000003044
Freq Check: 0.9999999999996956
Freq Check: 1.000000000000305
Freq Check: 0.9999999999997025
Freq Check: 1.0000000000003155
Freq Check: 0.9999999999997026
Freq Check: 1.0000000000003368
Freq Check: 0.9999999999996659
Freq Check: 1.0000000000003197
Freq Check: 0.9999999999996677
Freq Check: 1.0000000000003375
Freq Check: 0.9999999999994698
Freq Check: 1.000000000000522
Freq Check: 0.9999999999994488
Freq Check: 1.0000000000005338
Freq Check: 0.9999999999994298
Freq Check: 1.00000000000057
Freq Check: 0.999999999999292
Freq Check: 1.0000000000006724
Freq Check: 0.99999999999932
Freq Check: 1.0000

Freq Check: 1.0000000000000329
Freq Check: 0.9999999999999671
Freq Check: 1.0000000000000329
Freq Check: 0.9999999999999889
Freq Check: 0.9999999999999905
Freq Check: 1.0000000000000095
Freq Check: 0.9999999999999907
Freq Check: 1.0000000000000107
Freq Check: 0.9999999999999932
Freq Check: 1.0000000000000175
Freq Check: 0.9999999999999825
Freq Check: 1.0000000000000246
Freq Check: 0.9999999999999714
Freq Check: 1.0000000000000255
Freq Check: 0.9999999999998609
Freq Check: 0.9999999999999933
Freq Check: 0.9999999999996939
Freq Check: 1.000000000000174
Freq Check: 0.9999999999996277
Freq Check: 1.000000000000044
Freq Check: 0.9999999999997367
Freq Check: 0.9999999999997674
Freq Check: 1.0000000000005702
Freq Check: 0.9999999999963062
Freq Check: 1.0000000000035998
Freq Check: 0.9999999999963728
Freq Check: 1.0000000000032818
Freq Check: 0.9999999999966817
Freq Check: 1.0000000000032936
Freq Check: 0.9999999999966264
Freq Check: 1.0000000000033304
Freq Check: 0.9999999999965549
Freq Check

Freq Check: 1.000000000000214
Freq Check: 0.9999999999996863
Freq Check: 1.0000000000002733
Freq Check: 0.9999999999997216
Freq Check: 1.0000000000002784
Freq Check: 0.9999999999996864
Freq Check: 1.000000000000314
Freq Check: 0.999999999999686
Freq Check: 1.000000000000314
Freq Check: 0.9999999999996861
Freq Check: 1.0000000000002847
Freq Check: 0.9999999999997151
Freq Check: 1.0000000000002847
Freq Check: 0.9999999999997156
Freq Check: 1.0000000000002847
Freq Check: 0.9999999999997142
Freq Check: 1.0000000000002858
Freq Check: 0.9999999999997142
Freq Check: 1.0000000000002858
Freq Check: 0.9999999999997142
Freq Check: 1.0000000000002858
Freq Check: 0.9999999999997142
Freq Check: 1.0000000000002858
Freq Check: 0.9999999999997143
Freq Check: 0.9999999999997143
Freq Check: 1.0000000000002858
-------------------------------
Working on 16
Freq Check: 0.999999999999993
Freq Check: 1.0000000000000215
Freq Check: 0.9999999999999806
Freq Check: 0.9999999999999961
Freq Check: 0.999999999999997

Freq Check: 1.000000000000087
Freq Check: 0.9999999999999126
Freq Check: 1.0000000000000964
Freq Check: 0.9999999999998933
Freq Check: 1.0000000000001195
Freq Check: 0.9999999999998834
Freq Check: 1.000000000000091
Freq Check: 0.9999999999999096
Freq Check: 1.0000000000000864
Freq Check: 0.9999999999998438
Freq Check: 1.000000000000127
Freq Check: 0.9999999999997629
Freq Check: 1.0000000000001927
Freq Check: 0.9999999999996587
Freq Check: 1.0000000000001792
Freq Check: 0.9999999999997597
Freq Check: 1.0000000000000902
Freq Check: 0.9999999999999074
Freq Check: 0.9999999999999898
Freq Check: 0.9999999999999283
Freq Check: 0.9999999999999994
Freq Check: 0.9999999999998824
Freq Check: 1.0000000000000508
Freq Check: 0.9999999999999493
Freq Check: 1.0000000000000508
Freq Check: 0.999999999999949
Freq Check: 1.0000000000000513
Freq Check: 0.9999999999998623
Freq Check: 1.000000000000138
Freq Check: 0.9999999999998622
Freq Check: 1.0000000000001374
Freq Check: 0.9999999999998301
Freq Check: 1

...

Freq Check: 0.9999999999999762
Freq Check: 1.000000000000051
Freq Check: 0.9999999999999328
Freq Check: 1.0000000000000857
Freq Check: 0.9999999999998972
Freq Check: 1.0000000000000842
Freq Check: 0.9999999999999158
Freq Check: 1.0000000000000842
Freq Check: 0.9999999999999158
Freq Check: 1.0000000000000842
Freq Check: 0.9999999999999156
Freq Check: 1.0000000000000842
Freq Check: 0.9999999999999227
Freq Check: 1.000000000000079
Freq Check: 0.9999999999999212
Freq Check: 1.0000000000000895
Freq Check: 0.9999999999999262
Freq Check: 1.000000000000074
Freq Check: 0.9999999999999079
Freq Check: 1.0000000000001121
Freq Check: 0.9999999999998803
Freq Check: 1.0000000000001084
Freq Check: 0.9999999999998929
Freq Check: 1.0000000000001064
Freq Check: 0.9999999999999083
Freq Check: 1.0000000000000295
Freq Check: 0.9999999999998881
Freq Check: 0.9999999999994579
Freq Check: 1.0000000000002844
Freq Check: 0.9999999999994517
Freq Check: 1.0000000000002425
Freq Check: 0.9999999999997543
Freq Check:

Freq Check: 0.9999999999999776
Freq Check: 1.0000000000000264
Freq Check: 0.9999999999999598
Freq Check: 1.0000000000000402
Freq Check: 0.9999999999999598
Freq Check: 1.0000000000000402
Freq Check: 0.9999999999999598
Freq Check: 1.0000000000000402
Freq Check: 0.9999999999999607
Freq Check: 1.0000000000000397
Freq Check: 0.9999999999999754
Freq Check: 1.0000000000000215
Freq Check: 0.9999999999999846
Freq Check: 1.0000000000000182
Freq Check: 0.999999999999983
Freq Check: 1.000000000000019
Freq Check: 0.9999999999999805
Freq Check: 1.0000000000000164
Freq Check: 0.9999999999999835
Freq Check: 0.9999999999999586
Freq Check: 0.9999999999997675
Freq Check: 1.0000000000000673
Freq Check: 1.0000000000003766
Freq Check: 0.9999999999990508
Freq Check: 1.0000000000008993
Freq Check: 0.999999999999101
Freq Check: 1.0000000000008986
Freq Check: 0.9999999999991008
Freq Check: 1.000000000000899
Freq Check: 0.9999999999991
Freq Check: 1.0000000000010407
Freq Check: 0.999999999998756
Freq Check: 0.99

Freq Check: 0.9999999999997948
Freq Check: 1.000000000000205
Freq Check: 0.9999999999997952
Freq Check: 1.0000000000002047
Freq Check: 0.9999999999997361
-------------------------------
Working on 255
Freq Check: 0.9999999999999716
Freq Check: 1.0000000000000546
Freq Check: 0.9999999999999508
Freq Check: 1.0000000000000462
Freq Check: 0.9999999999999545
Freq Check: 0.9999999999998257
Freq Check: 1.0000000000003906
Freq Check: 0.9999999999995914
Freq Check: 1.0000000000004086
Freq Check: 0.9999999999995914
Freq Check: 1.0000000000004086
Freq Check: 0.9999999999995914
Freq Check: 1.0000000000004083
Freq Check: 0.9999999999995917
Freq Check: 0.9999999999993031
Freq Check: 1.0000000000018063
Freq Check: 0.9999999999980076
Freq Check: 1.0000000000010123
Freq Check: 0.9999999999988952
Freq Check: 1.0000000000011138
Freq Check: 1.000000000000137
Freq Check: 0.9999999999998641
Freq Check: 0.9999999999988188
Freq Check: 1.0000000000011813
Freq Check: 0.9999999999988186
Freq Check: 1.00000000000

Freq Check: 0.9999999999998023
Freq Check: 1.00000000000027
Freq Check: 0.9999999999995759
Freq Check: 1.0000000000003895
Freq Check: 0.9999999999994171
Freq Check: 1.0000000000005835
Freq Check: 0.9999999999994149
Freq Check: 1.0000000000005227
Freq Check: 0.9999999999991703
Freq Check: 1.0000000000008
Freq Check: 0.9999999999989838
Freq Check: 1.000000000001111
Freq Check: 0.999999999998708
Freq Check: 1.000000000001293
Freq Check: 0.9999999999987076
Freq Check: 1.000000000001217
Freq Check: 0.9999999999986927
Freq Check: 1.0000000000011524
Freq Check: 0.9999999999987864
Freq Check: 1.0000000000012101
Freq Check: 0.9999999999987383
Freq Check: 1.000000000001212
Freq Check: 0.9999999999987018
Freq Check: 1.0000000000010645
Freq Check: 0.9999999999989355
Freq Check: 1.0000000000010645
Freq Check: 0.9999999999989355
Freq Check: 1.0000000000010645
-------------------------------
Working on 260
Freq Check: 0.9999999999999702
Freq Check: 1.000000000000055
Freq Check: 0.9999999999999497
Fre

Freq Check: 1.0000000000000109
Freq Check: 0.9999999999998661
Freq Check: 1.0000000000000784
Freq Check: 0.9999999999996121
Freq Check: 1.0000000000002924
Freq Check: 0.9999999999994049
Freq Check: 1.0000000000002829
Freq Check: 0.9999999999993872
Freq Check: 1.0000000000003104
Freq Check: 0.9999999999994805
Freq Check: 1.0000000000002343
Freq Check: 0.999999999999628
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998855
Freq Check: 0.9999999999998854
Freq Check: 1.0000000000001146
Freq Check: 0.9999999999996774
Freq Check: 1.0000000000002764
Freq Check: 0.9999999999996343
Freq Check: 1.0000000000003526
Freq Check: 0.9999999999995874
Freq Check: 1.0000000000004123
Freq Check: 0.9999999999995747
Freq Check: 0.9999999999995747
Freq Chec

In [18]:
cfg.to_pickle(raw_prediction,'raw_prediction')

In [19]:
prediction = copy.deepcopy(raw_prediction)

In [20]:
def clean_prediction(pred):
    pred_freq = {n: 0 for n in test_replays['cluster'].unique()}
    for vote in pred:
        total = sum(vote[0].values())
        for key in vote[0].keys():
            pred_freq[key] += vote[0][key] / total * vote[1]
            
    max_key = max(pred_freq, key=pred_freq.get)
    return [max_key, min([pred_freq[max_key],1.0])]

In [21]:
df_list = []
for index,pred in enumerate(prediction):
    to_list = [pred[0],pred[1]] + clean_prediction(pred[2])
    df_list.append(to_list)

In [22]:
pred_df = pd.DataFrame(df_list)
pred_df.columns = ['unique','num_obs','max_cluster','bayes_prob']

pred_scoring_df = pd.merge(results_df,pred_df,left_index=True,right_on='unique')

In [23]:
right = pred_scoring_df[pred_scoring_df['cluster'] == pred_scoring_df['max_cluster']]
wrong = pred_scoring_df[pred_scoring_df['cluster'] != pred_scoring_df['max_cluster']]

In [24]:
right

,cluster,race,blind,unique,num_obs,max_cluster,bayes_prob
76,1,1,0.142632,"(sc_pvt, 4698)",49,1,1.000000
191,7,2,0.441491,"(sc_pvt, 3988)",52,7,0.587824
192,6,1,0.144996,"(sc_pvt, 1429)",35,6,0.993811
154,7,2,0.441491,"(sc_pvt, 1802)",53,7,0.654165
171,0,1,0.300630,"(sc_pvt, 1360)",44,0,0.915360
176,7,2,0.441491,"(sc_pvt, 1883)",52,7,0.528169
19,7,2,0.441491,"(sc_pvp, 597)",58,7,0.999956
72,8,2,0.239191,"(sc_pvt, 1116)",45,8,0.405210
48,0,1,0.300630,"(sc_pvt, 3106)",81,0,1.000000
189,6,1,0.144996,"(sc_pvt, 3986)",69,6,0.701740
